In [ ]:
from GeoValuator import DATA_DIR
import os
import yaml
import geopandas as gpd
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import glob
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import timm
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

In [ ]:
with open('download_checkpoint.json', 'r') as f:
    data = json.load(f)

# Create a district to bin mapping
district_bin_mapping = {}

for item in data['successful_downloads']:
    district = item['district']
    bin_id = item['price_bin']
    district_bin_mapping[district] = bin_id

# Use mapping to label your images
IMAGE_BASE_DIR = os.path.join(DATA_DIR, "processed", "images")

image_paths = []
image_labels = []
district_names_list = []

# Get all district folders
all_district_folders = [d for d in os.listdir(IMAGE_BASE_DIR) 
                       if os.path.isdir(os.path.join(IMAGE_BASE_DIR, d))]

# Load images and assign bin labels using the mapping
for district_folder in all_district_folders:
    if district_folder in district_bin_mapping:
        bin_id = district_bin_mapping[district_folder]
        district_image_dir = os.path.join(IMAGE_BASE_DIR, district_folder)
        
        # Get all images in this district folder
        pattern = os.path.join(district_image_dir, '*.jpg')
        image_files = glob.glob(pattern)
        
        for image_path in image_files:
            image_paths.append(image_path)
            image_labels.append(bin_id)
            district_names_list.append(district_folder)

# Create DataFrame
image_df = pd.DataFrame({
    'image_path': image_paths,
    'label': image_labels,
    'district': district_names_list
})

NUM_LABELS = 10

images = []

for i in range(NUM_LABELS):

    label_images = image_df[image_df['label'] == i]
    label_paths = label_images['image_path'].tolist()

# Group images by label
label_groups = {}
for label in image_df['label'].unique():
    label_groups[label] = image_df[image_df['label'] == label]

# Split each label separately
train_dfs, val_dfs, test_dfs = [], [], []

for label, group in label_groups.items():
    train_val, test = train_test_split(group, test_size=0.0654, random_state=42)
    train, val = train_test_split(train_val, test_size=0.07, random_state=42)
    
    train_dfs.append(train)
    val_dfs.append(val)
    test_dfs.append(test)

# Combine all labels
train_df = pd.concat(train_dfs).reset_index(drop=True)
val_df = pd.concat(val_dfs).reset_index(drop=True)
test_df = pd.concat(test_dfs).reset_index(drop=True)

District to bin mapping:
{'Britz': 0, 'Falkenhagener Feld': 0, 'Fennpfuhl': 0, 'Gropiusstadt': 0, 'Marienfelde': 0, 'Marzahn': 0, 'Neu-Hohenschönhausen': 0, 'Staaken': 0, 'Baumschulenweg': 1, 'Biesdorf': 1, 'Buckow': 1, 'Friedrichshagen': 1, 'Lichtenrade': 1, 'Mariendorf': 1, 'Wittenau': 1, 'Buch': 2, 'Charlottenburg-Nord': 2, 'Friedenau': 3, 'Hellersdorf': 2, 'Karow': 2, 'Lankwitz': 2, 'Lichtenberg': 2, 'Tempelhof': 2, 'Wilhelmstadt': 2, 'Johannisthal': 3, 'Nikolassee': 3, 'Rahnsdorf': 3, 'Rudow': 3, 'Spandau': 3, 'Tegel': 3, 'Alt-Hohenschönhausen': 4, 'Friedrichsfelde': 4, 'Grünau': 4, 'Hakenfelde': 5, 'Heiligensee': 4, 'Niederschönhausen': 4, 'Reinickendorf': 4, 'Siemensstadt': 4, 'Französisch Buchholz': 5, 'Frohnau': 5, 'Kaulsdorf': 5, 'Mahlsdorf': 5, 'Schmöckwitz': 5, 'Wedding': 5, 'Zehlendorf': 5, 'Alt-Treptow': 6, 'Altglienicke': 6, 'Gesundbrunnen': 6, 'Karlshorst': 6, 'Kreuzberg': 6, 'Lichterfelde': 6, 'Oberschöneweide': 6, 'Heinersdorf': 7, 'Köpenick': 7, 'Neukölln': 7, 'Panko

In [ ]:
class StreetViewDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label']
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Define transforms for B3 (300x300)
INPUT_SIZE = 300

train_transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Use ImageNet means and std
])

val_transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = StreetViewDataset(train_df, transform=train_transform)
val_dataset = StreetViewDataset(val_df, transform=val_transform)
test_dataset = StreetViewDataset(test_df, transform=val_transform)

# Create dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

DataLoaders created successfully!
Train batches: 547
Val batches: 42
Test batches: 42


In [ ]:
# GPU Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Classification Model (für deine 10 Bins)
class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes=10, model_name='efficientnet_b3'):
        super(EfficientNetClassifier, self).__init__()
        
        self.backbone = timm.create_model(model_name, 
                                         pretrained=True,
                                         num_classes=0,
                                         global_pool='avg')
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.backbone.num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)

# Put model on GPU
model = EfficientNetClassifier(num_classes=10, model_name='efficientnet_b3')
model = model.to(device)

print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Model is on: {next(model.parameters()).device}")

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, scheduler=None):
    model = model.to(device)
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    best_val_accuracy = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        
        for images, labels in train_pbar:
            images = images.to(device)
            labels = labels.long().to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
            running_loss += loss.item() * images.size(0)
            
            # Update progress bar with current metrics
            current_acc = 100 * correct_train / total_train
            train_pbar.set_postfix({
                'Loss': loss.item(),
                'Acc': f'{current_acc:.2f}%'
            })
        
        # Calculate epoch training metrics
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct_train / total_train
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.long().to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                
                # Calculate validation accuracy
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        # Calculate epoch validation metrics
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * correct_val / total_val
        
        # Store metrics
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accuracies.append(epoch_train_acc)
        val_accuracies.append(epoch_val_acc)
        
        # Step scheduler
        if scheduler:
            scheduler.step()
        
        # Print detailed epoch summary
        print(f'\nEpoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}%')
        print(f'  Val Loss:   {epoch_val_loss:.4f} | Val Acc:   {epoch_val_acc:.2f}%')
        
        # Save best model based on validation accuracy
        if epoch_val_acc > best_val_accuracy:
            best_val_accuracy = epoch_val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'  ↳ New best model saved! (Val Acc: {epoch_val_acc:.2f}%)')
    
    return train_losses, val_losses, train_accuracies, val_accuracies

def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot losses
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Val Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Plot accuracies
    ax2.plot(train_accuracies, label='Train Accuracy')
    ax2.plot(val_accuracies, label='Val Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Use CrossEntropy Loss für Classification
criterion = nn.CrossEntropyLoss()

# Stage 1: Feature Extraction
print("=== Stage 1: Feature Extraction ===")

for param in model.backbone.parameters():
    param.requires_grad = False

optimizer_stage1 = optim.Adam(model.classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Train with metrics
train_losses_s1, val_losses_s1, train_acc_s1, val_acc_s1 = train_model(
    model, train_loader, val_loader, criterion, optimizer_stage1, 
    num_epochs=10
)

# Stage 2: Fine-Tuning
print("\n=== Stage 2: Fine-Tuning ===")

for param in model.backbone.parameters():
    param.requires_grad = True

optimizer_stage2 = optim.Adam(
    [{'params': model.classifier.parameters(), 'lr': 1e-4},
     {'params': model.backbone.parameters(), 'lr': 1e-5}]
)

scheduler = StepLR(optimizer_stage2, step_size=5, gamma=0.1)

train_losses_s2, val_losses_s2, train_acc_s2, val_acc_s2 = train_model(
    model, train_loader, val_loader, criterion, optimizer_stage2, 
    num_epochs=20, scheduler=scheduler
)

# Plot metrics
print("\n=== Stage 1 Metrics ===")
plot_metrics(train_losses_s1, val_losses_s1, train_acc_s1, val_acc_s1)

print("\n=== Stage 2 Metrics ===")
plot_metrics(train_losses_s2, val_losses_s2, train_acc_s2, val_acc_s2)

# Combine all metrics for full training view
full_train_losses = train_losses_s1 + train_losses_s2
full_val_losses = val_losses_s1 + val_losses_s2
full_train_accs = train_acc_s1 + train_acc_s2
full_val_accs = val_acc_s1 + val_acc_s2

print("\n=== Full Training Metrics ===")
plot_metrics(full_train_losses, full_val_losses, full_train_accs, full_val_accs)

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    model = model.to(device)
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.long().to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Final Evaluation
print("\n=== Final Evaluation ===")
model.load_state_dict(torch.load('best_model.pth'))
accuracy = evaluate_model(model, test_loader)